# Introduction

This notebook demonstrates the use of the Materials API using Python. We will do so with Python Materials Genomics (pymatgen)'s high level tools as well as using the requests package.

# Using pymatgen's MPRester

In [2]:
# We start by importing MPRester, which is available from the root import of pymatgen.
from pymatgen import MPRester

# Initializing MPRester. Note that you can call MPRester. MPRester looks for the API key in two places: 
# - Supplying it directly as an __init__ arg.
# - Setting the "MAPI_KEY" environment variable.
# Please obtain your API key at https://www.materialsproject.org/dashboard

m = MPRester()

## Doing simple queries using the high-level methods.

In [4]:
data = m.query(criteria={"task_id": "mp-1234"}, properties=["final_energy"])
print data

[{u'final_energy': -26.94736193}]


In [6]:
data = m.query(criteria={"pretty_formula": "Li2O"}, properties=["spacegroup.symbol"])
print data

[{u'spacegroup.symbol': u'Fm-3m'}, {u'spacegroup.symbol': u'Pnma'}]


In [7]:
data = m.query(criteria={"elements": {"$all": ["Fe", "O"]}}, properties=["exp.tags", "icsd_id"])
print data

[{u'icsd_id': 415251, u'exp.tags': [u'Iron(III) oxide', u'Diiron(III) oxide - alpha', u'Iron(III) oxide - alpha', u'Iron(III) oxide - alpha HP1', u'Diiron(III) oxide - alpha, mesoporous', u'Hematite']}, {u'icsd_id': 158504, u'exp.tags': [u'Iron diiron(III) oxide - LT', u'Magnetite (Cr-bearing)', u'Triiron tetraoxide', u'Magnetite (annealedat 1373 K for 7 d)', u'Iron diiron(III) oxide - HT', u'Iron(II) diiron(III) tetraoxide', u'Triiron(II/III) tetraoxide', u'Iron oxide (3/4)', u'Magnetite low', u'Magnetite ( annealed at 1173 K for 28 h)', u'Magnetite', u'Diiron(III) iron oxide', u'Iron diiron(III) oxide', u'Iron(II) diiron(III) oxide']}, {u'icsd_id': None, u'exp.tags': [u'Wuestite', u'Iron oxide - HT', u'Iron oxide - alpha', u'Ferrous oxide - HT', u'Iron oxide']}, {u'icsd_id': 51122, u'exp.tags': [u'Diiron(III) oxide - epsilon, nanocrystal', u'Diiron(III) oxide - epsilon', u'Iron(III) oxide - epsilon']}, {u'icsd_id': 35001, u'exp.tags': [u'Iron diiron(III) oxide - LT', u'Iron diiron(II